# 0/1 Knapsack problem

In [1]:
function ks(i, w, wt, vl)
    if i == 0
        0
    elseif w == 0
        0
    elseif wt[i] > w
        ks(i - 1, w, wt, vl)
    else
        r1 = ks(i - 1, w - wt[i], wt, vl)
        r2 = ks(i - 1, w, wt, vl)
        max(vl[i] + r1, r2)
    end
end

ks(w, wt, vl) = ks(length(wt), w, wt, vl)

ks (generic function with 2 methods)

In [2]:
ks(50, (10, 20, 30), (100, 200, 300))

500

## Специализация по `w` и `wt`

In [3]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

In [4]:
function ks_gen_impl1(i, w, wt)
    if i == 0
        0
    elseif w == 0
        0
    elseif wt[i] > w
        ks_gen_impl1(i - 1, w, wt)
    else
        r1 = ks_gen_impl1(i - 1, w - wt[i], wt)
        r2 = ks_gen_impl1(i - 1, w, wt)
        :(max(vl[$i] + $r1, $r2))
    end
end

ks_gen_impl1(w, wt) = ks_gen_impl1(length(wt), w, wt)

ks_gen_impl1 (generic function with 2 methods)

In [5]:
ks_gen_impl1(50, (10, 20, 30)) |> cleanup

:(max(vl[3] + max(vl[2] + 0, max(vl[1] + 0, 0)), max(vl[2] + max(vl[1] + 0, 0), max(vl[1] + 0, 0))))

## Мемоизация

In [6]:
function g_ass!(es, d, s::Symbol, c::Int64)
    d[s] = c
end

function g_ass!(es, d, s::Symbol, u::Symbol)
    d[s] = u
end

function g_ass!(es, d, s::Symbol, e)
    haskey(d, s) && return
    d[s] = s
    push!(es, :($s = $e))
end

g_ass! (generic function with 3 methods)

In [7]:
function ks_gen_impl2(es, d, i, w, wt)
    r = Symbol("r_", i, "_", w)

    if haskey(d, r)
        return d[r]
    end

    if i == 0
        g_ass!(es, d, r, 0)
    elseif w == 0
        g_ass!(es, d, r, 0)
    elseif wt[i] > w
        g_ass!(es, d, r, ks_gen_impl2(es, d, i - 1, w, wt))
    else
        r1 = ks_gen_impl2(es, d, i - 1, w - wt[i], wt)
        r2 = ks_gen_impl2(es, d, i - 1, w, wt)
        g_ass!(es, d, r, :(max(vl[$i] + $r1, $r2)))
    end
    return d[r]
end

ks_gen_impl2 (generic function with 1 method)

In [8]:
function ks_gen_impl2(w, wt)
    es = Expr[]
    d = Dict{Symbol,Any}()
    r = ks_gen_impl2(es, d, length(wt), w, wt)

    quote
        $(es...)
        return $r
    end
end

ks_gen_impl2 (generic function with 2 methods)

In [9]:
ks_gen_impl2(50, (10, 20, 30)) |> cleanup

quote
    r_1_20 = max(vl[1] + 0, 0)
    r_2_20 = max(vl[2] + 0, r_1_20)
    r_1_30 = max(vl[1] + 0, 0)
    r_1_50 = max(vl[1] + 0, 0)
    r_2_50 = max(vl[2] + r_1_30, r_1_50)
    r_3_50 = max(vl[3] + r_2_20, r_2_50)
    return r_3_50
end

In [10]:
add_s(x, y::Int64) =
    if y == 0
        x
    else
        :($x + $y)
    end

add_s(x, y) = :($x + $y)

add_s (generic function with 2 methods)

In [11]:
max_s(x, y::Int64) =
    if y == 0
        x
    else
        :(max($x, $y))
    end

max_s(x, y) = :(max($x, $y))

max_s (generic function with 2 methods)

In [12]:
function ks_gen_impl3(es, d, i, w, wt)
    r = Symbol("r_", i, "_", w)

    if haskey(d, r)
        return d[r]
    end

    if i == 0
        g_ass!(es, d, r, 0)
    elseif w == 0
        g_ass!(es, d, r, 0)
    elseif wt[i] > w
        g_ass!(es, d, r, ks_gen_impl3(es, d, i - 1, w, wt))
    else
        r1 = ks_gen_impl3(es, d, i - 1, w - wt[i], wt)
        r2 = ks_gen_impl3(es, d, i - 1, w, wt)
        vl_i = Symbol("vl_", i)
        g_ass!(es, d, vl_i, :(vl[$i]))
        g_ass!(es, d, r, max_s(add_s(vl_i, r1), r2))
    end
    return d[r]
end

ks_gen_impl3 (generic function with 1 method)

In [13]:
function ks_gen_impl3(w, wt)
    es = Expr[]
    d = Dict{Symbol,Any}()
    r = ks_gen_impl3(es, d, length(wt), w, wt)

    quote
        $(es...)
        return $r
    end
end

ks_gen_impl3 (generic function with 2 methods)

In [14]:
ks_gen_impl3(50, (10, 20, 30)) |> cleanup

quote
    vl_1 = vl[1]
    vl_2 = vl[2]
    r_2_20 = max(vl_2, vl_1)
    r_2_50 = max(vl_2 + vl_1, vl_1)
    vl_3 = vl[3]
    r_3_50 = max(vl_3 + r_2_20, r_2_50)
    return r_3_50
end

In [15]:
ks_gen_impl3(60, (70, 10, 50, 20, 30, 40, 60)) |> cleanup

quote
    vl_2 = vl[2]
    vl_4 = vl[4]
    r_4_20 = max(vl_4, vl_2)
    r_4_30 = max(vl_4 + vl_2, vl_2)
    vl_3 = vl[3]
    r_3_60 = max(vl_3 + vl_2, vl_2)
    r_4_60 = max(vl_4 + vl_2, r_3_60)
    vl_5 = vl[5]
    r_5_60 = max(vl_5 + r_4_30, r_4_60)
    vl_6 = vl[6]
    r_6_60 = max(vl_6 + r_4_20, r_5_60)
    vl_7 = vl[7]
    r_7_60 = max(vl_7, r_6_60)
    return r_7_60
end

In [16]:
@generated function ks_gen(::Val{w}, ::Val{wl}, vl) where {w,wl}
    ks_gen_impl3(w, wl)
end

ks_gen (generic function with 1 method)

In [17]:
ks_gen(Val(50), Val((10, 20, 30)), (100, 200, 300))

500